In [ ]:
import pypianoroll
import numpy as np
from PIL import Image
import glob, os

In [ ]:
CHUNK = 128 * 3

In [ ]:
def is_divisible_by(pianoroll, N=CHUNK):
    return pianoroll.shape[1] % N == 0

In [ ]:
def do_padding(pianoroll):
    x = CHUNK - (pianoroll.shape[1] % CHUNK)
    padding = np.zeros((pianoroll.shape[0], x))
    return np.concatenate((pianoroll, padding), axis=1)

# Maestro dataset

In [ ]:
RAW_DATA_PATH = "C://Users/mateu/Desktop/INZYNIERKA/data/midi/maestro-v3.0.0"
idx = 0
for midi_file in [y for x in os.walk(RAW_DATA_PATH) for y in glob.glob(os.path.join(x[0], '*.midi'))]:
    multitrack = pypianoroll.read(midi_file)
    multitrack.binarize()
    pr_array = multitrack.tracks[0].pianoroll
    pr_T = np.transpose(pr_array)
    x = do_padding(pr_T)
    batches = int(x.shape[1] / CHUNK)
    split_pr = np.array_split(x, batches, axis=1)
    for chunk in split_pr:
        data = Image.fromarray(chunk.astype(bool))
        data.save(f'C://Users/mateu/Desktop/INZYNIERKA/train_dataset/train/{idx}_train.bmp')
        idx += 1

# Lakh Pianoroll Dataset (LPD) - cleansed

In [ ]:
def how_much_music_data(array):
    array = array.flatten().astype(int)
    return sum(array) / len(array)

In [ ]:
def is_good_sample(array, percentage_of_music_data_recquired):
    return how_much_music_data(array) >= percentage_of_music_data_recquired

In [ ]:
def is_empty(array):
    return np.all(array == False)

In [ ]:
LPD_PATH = 'D:/LPD_5'
idx = 0

DO_DATA_FILTRATION = True
FILTRATION_THRESHOLD = 0.03

for midi_file in [y for x in os.walk(LPD_PATH) for y in glob.glob(os.path.join(x[0], '*.npz'))]:
    multitrack = pypianoroll.load(midi_file)
    multitrack.binarize()
    # Take Piano which has index [1]
    pr_array = multitrack.tracks[1].pianoroll
    pr_T = np.transpose(pr_array)
    x = do_padding(pr_T)
    batches = int(x.shape[1] / CHUNK)
    split_pr = np.array_split(x, batches, axis=1)
    for chunk in split_pr:
        if is_empty(chunk):
            continue
        if DO_DATA_FILTRATION and not is_good_sample(chunk, FILTRATION_THRESHOLD):
            continue
        data = Image.fromarray(chunk.astype(bool))

        if DO_DATA_FILTRATION:
            data.save(f'C://Users/mateu/Desktop/INZYNIERKA/train_dataset_lpd_filtered_3_sequence/train/{idx}_train.bmp')
        else:
            data.save(f'C://Users/mateu/Desktop/INZYNIERKA/train_dataset_lpd/train/{idx}_train.bmp')
        idx += 1